In [6]:
#!pip install newsapi-python

In [18]:
from newsapi import NewsApiClient
import os
api_key = "####"
import datetime as dt 
import pandas as pd
from typing import Dict, List, Any
from newsapi import NewsApiClient
import datetime as dt 

In [20]:
newsapi = NewsApiClient(api_key=api_key)

In [22]:
data = newsapi.get_everything(q='deepseek', from_param='2025-01-09')
print(data['totalResults'])

16541


In [9]:
#data

## Deepseek Data Collection - Day One

In [61]:
# References can be found from here --> https://www.youtube.com/watch?v=oUuIVJ96JsI

def news_to_dataframe(data: Dict[str, Any]) -> pd.DataFrame:
    """
    Convert news API response data into a pandas DataFrame.
    
    Args:
        news_data (dict): Dictionary containing news API response with 'articles' key
        
    Returns:
        pd.DataFrame: DataFrame containing processed news articles data
    """
    # Extract articles list
    articles = data.get('articles', [])
    
    # Process each article to flatten the structure
    processed_articles = []
    
    for article in articles:
        processed_article = {
            'source_id': article['source'].get('id'),
            'source_name': article['source'].get('name'),
            'author': article.get('author'),
            'title': article.get('title'),
            'description': article.get('description'),
            'url': article.get('url'),
            'urlToImage': article.get('urlToImage'),
            'publishedAt': pd.to_datetime(article.get('publishedAt')),
            'content': article.get('content')
        }
        processed_articles.append(processed_article)
    
    # Create DataFrame
    df = pd.DataFrame(processed_articles)
    
    return df

df = news_to_dataframe(data)
df

,source_id,source_name,author,title,description,url,urlToImage,publishedAt,content
0,None,Yahoo Entertainment,Steve Dent,China’s DeepSeek AI assistant becomes top free...,Chinese AI assistant DeepSeek has become the t...,https://consent.yahoo.com/v2/collectConsent?se...,None,2025-01-27 13:44:45+00:00,"If you click 'Accept all', we and our partners..."
1,None,BBC News,None,Is China's AI tool DeepSeek as good as it seems?,The artificial intelligence (AI) tool has shoc...,https://www.bbc.com/news/articles/cx2jxvn0r51o,https://ichef.bbci.co.uk/news/1024/branded_new...,2025-01-27 20:11:50+00:00,"DeepSeek, a Chinese AI-chatbot app which launc..."
2,None,Gizmodo.com,Lucas Ropek,DeepSeek Releases Open-Source AI Image Generat...,Silicon Valley's Chinese competitor has releas...,https://gizmodo.com/deepseek-releases-open-sou...,https://gizmodo.com/app/uploads/2025/01/DeepSe...,2025-01-27 21:40:47+00:00,"DeepSeek, the Chinese startup that has managed..."
3,None,Android Central,bradypsnyder@gmail.com (Brady Snyder),Data Protection Day is a great day to stop usi...,DeepSeek's new AI models have taken the world ...,https://www.androidcentral.com/apps-software/d...,https://cdn.mos.cms.futurecdn.net/NYR57UzpFC38...,2025-01-28 17:41:36+00:00,"Data Protection Day is today, Jan. 28, 2025, a..."
4,None,Android Central,harish.jonnalagadda@futurenet.com (Harish Jonn...,DeepSeek already had a $1 trillion impact — an...,DeepSeek is showing the world that the latest ...,https://www.androidcentral.com/apps-software/d...,https://cdn.mos.cms.futurecdn.net/ceAoG3XT8se7...,2025-01-28 15:36:09+00:00,The tech world is collectively losing its mind...
...,...,...,...,...,...,...,...,...,...
94,None,Genbeta.com,Marcos Merino,"Un jefe de OpenAI ve ""exagerada"" la repercusió...",La irrupción de DeepSeek y su modelo DeepSeek ...,https://www.genbeta.com/inteligencia-artificia...,https://i.blogs.es/683d41/9405b09a9b80579f281d...,2025-01-28 21:06:52+00:00,La irrupción de DeepSeek y su modelo DeepSeek ...
95,None,Genbeta.com,José Alberto Lizana,"DeepSeek: qué es, cómo funciona y todo lo que ...","El modelo de inteligencia artificial DeepSeek,...",https://www.genbeta.com/a-fondo/deepseek-que-c...,https://i.blogs.es/ea130b/deepseek-logo/840_56...,2025-01-27 13:11:40+00:00,"El modelo de inteligencia artificial DeepSeek,..."
96,None,heise online,Eva-Maria Weiß,Deepseek veröffentlicht multimodales Modell Ja...,Noch mehr Konkurrenz für das Silicon Valley: D...,https://www.heise.de/news/Deepseek-veroeffentl...,https://heise.cloudimg.io/bound/1200x1200/q85....,2025-01-28 07:06:00+00:00,"Das chinesische KI-Unternehmen Deepseek, das g..."
97,None,Genbeta.com,José Alberto Lizana,Enfrentamos a DeepSeek contra ChatGPT: sorpren...,Estamos viviendo un momento apabullante de la ...,https://www.genbeta.com/a-fondo/enfrentamos-a-...,https://i.blogs.es/87cd61/comparativa-/840_560...,2025-01-27 11:32:19+00:00,Estamos viviendo un momento apabullante de la ...


In [63]:
df.to_csv('deepseek_df.csv')

## Deepseek Data Collection - Day Two

In [1]:
# Resources can be found from here --> ChatGPT was utilized for this section.

newsapi = NewsApiClient(api_key="####")  

downloaded_file = 'deepseek_df.csv'

# Function to check and load previously downloaded articles (URLs)
def load_downloaded_articles(file_name: str = downloaded_file) -> set:
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
        return set(df['url'])  
    else:
        return set()  

# Function to store new articles' URLs in a CSV
def store_downloaded_articles(new_urls: list, file_name: str = downloaded_file):
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        df = pd.DataFrame(columns=['url'])
    
    # Add new URLs to the DataFrame and save
    new_df = pd.DataFrame({'url': new_urls})
    df = pd.concat([df, new_df]).drop_duplicates(subset=['url'], keep='last')
    df.to_csv(file_name, index=False)

# Function to convert the NewsAPI data to a DataFrame
def news_to_dataframe(data: Dict[str, Any]) -> pd.DataFrame:
    articles = data.get('articles', [])
    processed_articles = []
    
    for article in articles:
        processed_article = {
            'source_id': article['source'].get('id'),
            'source_name': article['source'].get('name'),
            'author': article.get('author'),
            'title': article.get('title'),
            'description': article.get('description'),
            'url': article.get('url'),
            'urlToImage': article.get('urlToImage'),
            'publishedAt': pd.to_datetime(article.get('publishedAt')),
            'content': article.get('content')
        }
        processed_articles.append(processed_article)
    
    df = pd.DataFrame(processed_articles)
    return df

# Function to fetch and check if there are new articles
def fetch_news(query: str, from_date: str) -> pd.DataFrame:
    # Check the total number of results first to determine if there's new data
    data = newsapi.get_everything(q=query, from_param=from_date)
    total_results = data['totalResults']
    
    # If no new articles or less than 1,000 articles, return early
    if total_results == 0:
        print("No new articles available.")
        return pd.DataFrame()  # Return empty DataFrame
    
    downloaded_urls = load_downloaded_articles()
    new_articles = [article for article in data['articles'] if article['url'] not in downloaded_urls]
    
    # If no new articles, return empty DataFrame
    if len(new_articles) == 0:
        print("No new articles to download.")
        return pd.DataFrame()
    
    # Convert new articles to DataFrame
    new_df = news_to_dataframe({'articles': new_articles})
    
    # Save URLs of new articles to avoid duplicates
    new_urls = [article['url'] for article in new_articles]
    store_downloaded_articles(new_urls)
    
    return new_df

query = 'deepseek'
from_date = (dt.datetime.now() - dt.timedelta(days=1)).strftime('%Y-%m-%d')  # Get articles from yesterday
new_data = fetch_news(query, from_date)

# Show and save the new data if available
if not new_data.empty:
    print(new_data)
    new_data.to_csv('Deepseek_new_article.csv', index=False)


           source_id          source_name  \
0               None  Yahoo Entertainment   
1               None          Gizmodo.com   
2               None            MacRumors   
3   business-insider     Business Insider   
4   business-insider     Business Insider   
..               ...                  ...   
94              None         Quartz India   
95              None         Caschys Blog   
96              None            Frandroid   
97              None       Les Numériques   
98              None            Frandroid   

                                     author  \
0                             Mariella Moon   
1                             Todd Feathers   
2                              Tim Hardwick   
3                            Ashley Stewart   
4   tspirlet@insider.com (Thibault Spirlet)   
..                                      ...   
94                                Bruce Gil   
95                                     Olli   
96                          Maxence 

## Deepseek Data Collection - Day Three

In [40]:
newsapi = NewsApiClient(api_key="####")

# Define the two downloaded files
downloaded_file_1 = 'Deepseek_Day_One.csv'
downloaded_file_2 = 'Deepseek_Day_Two.csv'

# Function to load previously downloaded article URLs from multiple files
def load_downloaded_articles(file_names: List[str]) -> set:
    urls = set()
    for file_name in file_names:
        if os.path.exists(file_name):
            df = pd.read_csv(file_name)
            urls.update(set(df['url']))
    return urls

# Function to store new article URLs in a CSV file
def store_downloaded_articles(new_urls: List[str], file_name: str):
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        df = pd.DataFrame(columns=['url'])
    
    new_df = pd.DataFrame({'url': new_urls})
    df = pd.concat([df, new_df]).drop_duplicates(subset=['url'], keep='last')
    df.to_csv(file_name, index=False)

# Function to convert NewsAPI data to a DataFrame
def news_to_dataframe(data: Dict[str, Any]) -> pd.DataFrame:
    articles = data.get('articles', [])
    processed_articles = []
    for article in articles:
        processed_article = {
            'source_id': article['source'].get('id'),
            'source_name': article['source'].get('name'),
            'author': article.get('author'),
            'title': article.get('title'),
            'description': article.get('description'),
            'url': article.get('url'),
            'urlToImage': article.get('urlToImage'),
            'publishedAt': pd.to_datetime(article.get('publishedAt')),
            'content': article.get('content')
        }
        processed_articles.append(processed_article)
    
    df = pd.DataFrame(processed_articles)
    return df

# Function to fetch new articles based on a query and date,
# while filtering out articles that have already been downloaded.
def fetch_news(query: str, from_date: str) -> pd.DataFrame:
    # Fetch data from NewsAPI
    data = newsapi.get_everything(q=query, from_param=from_date)
    total_results = data['totalResults']
    
    # If there are no results, return early.
    if total_results == 0:
        print("No new articles available.")
        return pd.DataFrame()  # Return an empty DataFrame
    
    # Load URLs from both downloaded files.
    downloaded_urls = load_downloaded_articles([downloaded_file_1, downloaded_file_2])
    # Filter for new articles (by URL)
    new_articles = [article for article in data['articles'] if article['url'] not in downloaded_urls]
    
    # If there are no new articles, return early.
    if not new_articles:
        print("No new articles to download.")
        return pd.DataFrame()
    
    # Convert the new articles to a DataFrame.
    new_df = news_to_dataframe({'articles': new_articles})
    
    # Save the new article URLs.
    new_urls = [article['url'] for article in new_articles]
    # Here we choose to update one of the files (e.g., downloaded_file_2). Adjust as needed.
    store_downloaded_articles(new_urls, downloaded_file_2)
    
    return new_df

query = 'deepseek'
# Get articles from yesterday
from_date = (dt.datetime.now() - dt.timedelta(days=1)).strftime('%Y-%m-%d')
new_data = fetch_news(query, from_date)

# Show and save the new data if available
if not new_data.empty:
    print(new_data)
    new_data.to_csv('Deepseek_Day_Three.csv', index=False)


           source_id          source_name                 author  \
0   business-insider     Business Insider             Jane Zhang   
1               None           Xataka.com        Yúbal Fernández   
2   business-insider     Business Insider            Tim Paradis   
3               None          Genbeta.com  Eva Rodriguez de Luis   
4               None         Slashdot.org            EditorDavid   
..               ...                  ...                    ...   
81              None         Weeraman.com      Anuradha Weeraman   
82       abc-news-au        ABC News (AU)            Alan Kohler   
83              None      Overclockers.ru          Алексей Сычёв   
84              None      Overclockers.ru              Madarator   
85              None  Yahoo Entertainment                   None   

                                                title  \
0   I'm an AI startup founder. Here are the 3 reas...   
1   Guía DeepSeek: 36 funciones y cosas que puedes...   
2           

## Deepseek Data Collection - Day Four

In [19]:
newsapi = NewsApiClient(api_key="####")

# Define the two downloaded files
downloaded_file_1 = 'Deepseek_Day_One.csv'
downloaded_file_2 = 'Deepseek_Day_Two.csv'
downloaded_file_3 = 'Deepseek_Day_Three.csv'

# Function to load previously downloaded article URLs from multiple files
def load_downloaded_articles(file_names: List[str]) -> set:
    urls = set()
    for file_name in file_names:
        if os.path.exists(file_name):
            df = pd.read_csv(file_name)
            urls.update(set(df['url']))
    return urls

# Function to store new article URLs in a CSV file
def store_downloaded_articles(new_urls: List[str], file_name: str):
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        df = pd.DataFrame(columns=['url'])
    
    new_df = pd.DataFrame({'url': new_urls})
    df = pd.concat([df, new_df]).drop_duplicates(subset=['url'], keep='last')
    df.to_csv(file_name, index=False)

# Function to convert NewsAPI data to a DataFrame
def news_to_dataframe(data: Dict[str, Any]) -> pd.DataFrame:
    articles = data.get('articles', [])
    processed_articles = []
    for article in articles:
        processed_article = {
            'source_id': article['source'].get('id'),
            'source_name': article['source'].get('name'),
            'author': article.get('author'),
            'title': article.get('title'),
            'description': article.get('description'),
            'url': article.get('url'),
            'urlToImage': article.get('urlToImage'),
            'publishedAt': pd.to_datetime(article.get('publishedAt')),
            'content': article.get('content')
        }
        processed_articles.append(processed_article)
    
    df = pd.DataFrame(processed_articles)
    return df

# Function to fetch new articles based on a query and date,
# while filtering out articles that have already been downloaded.
def fetch_news(query: str, from_date: str) -> pd.DataFrame:
    # Fetch data from NewsAPI
    data = newsapi.get_everything(q=query, from_param=from_date)
    total_results = data['totalResults']
    
    # If there are no results, return early.
    if total_results == 0:
        print("No new articles available.")
        return pd.DataFrame()  # Return an empty DataFrame
    
    # Load URLs from both downloaded files.
    downloaded_urls = load_downloaded_articles([downloaded_file_1, downloaded_file_2])
    # Filter for new articles (by URL)
    new_articles = [article for article in data['articles'] if article['url'] not in downloaded_urls]
    
    # If there are no new articles, return early.
    if not new_articles:
        print("No new articles to download.")
        return pd.DataFrame()
    
    # Convert the new articles to a DataFrame.
    new_df = news_to_dataframe({'articles': new_articles})
    
    # Save the new article URLs.
    new_urls = [article['url'] for article in new_articles]
    # Here we choose to update one of the files (e.g., downloaded_file_2). Adjust as needed.
    store_downloaded_articles(new_urls, downloaded_file_2)
    
    return new_df

query = 'deepseek'
# Get articles from yesterday
from_date = (dt.datetime.now() - dt.timedelta(days=1)).strftime('%Y-%m-%d')
new_data = fetch_news(query, from_date)

# Show and save the new data if available
if not new_data.empty:
    print(new_data)
    new_data.to_csv('Deepseek_Day_Four.csv', index=False)


           source_id          source_name                            author  \
0   business-insider     Business Insider                   Dan DeFrancesco   
1               None             Hackaday                       Donald Papp   
2               None          Gizmodo.com                     Todd Feathers   
3               None           Xataka.com                 Juan Carlos López   
4               None       The New Yorker                      John Cassidy   
..               ...                  ...                               ...   
90              None               Futura  Edward Back, Journaliste hi-tech   
91              None  Olhardigital.com.br           Leandro Costa Criscuolo   
92              None         1pezeshk.com                      علیرضا مجیدی   
93              None             TechSpot                       Rob Thubron   
94              None              Sapo.pt                    Ana Sofia Neto   

                                                tit

## Deepseek Data Collection - Day Five

In [14]:
newsapi = NewsApiClient(api_key="####")

# Define the two downloaded files
downloaded_file_1 = 'Deepseek_Day_One.csv'
downloaded_file_2 = 'Deepseek_Day_Two.csv'
downloaded_file_3 = 'Deepseek_Day_Three.csv'
downloaded_file_4 = 'Deepseek_Day_Four.csv'

# Function to load previously downloaded article URLs from multiple files
def load_downloaded_articles(file_names: List[str]) -> set:
    urls = set()
    for file_name in file_names:
        if os.path.exists(file_name):
            df = pd.read_csv(file_name)
            urls.update(set(df['url']))
    return urls

# Function to store new article URLs in a CSV file
def store_downloaded_articles(new_urls: List[str], file_name: str):
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        df = pd.DataFrame(columns=['url'])
    
    new_df = pd.DataFrame({'url': new_urls})
    df = pd.concat([df, new_df]).drop_duplicates(subset=['url'], keep='last')
    df.to_csv(file_name, index=False)

# Function to convert NewsAPI data to a DataFrame
def news_to_dataframe(data: Dict[str, Any]) -> pd.DataFrame:
    articles = data.get('articles', [])
    processed_articles = []
    for article in articles:
        processed_article = {
            'source_id': article['source'].get('id'),
            'source_name': article['source'].get('name'),
            'author': article.get('author'),
            'title': article.get('title'),
            'description': article.get('description'),
            'url': article.get('url'),
            'urlToImage': article.get('urlToImage'),
            'publishedAt': pd.to_datetime(article.get('publishedAt')),
            'content': article.get('content')
        }
        processed_articles.append(processed_article)
    
    df = pd.DataFrame(processed_articles)
    return df

# Function to fetch new articles based on a query and date,
# while filtering out articles that have already been downloaded.
def fetch_news(query: str, from_date: str) -> pd.DataFrame:
    # Fetch data from NewsAPI
    data = newsapi.get_everything(q=query, from_param=from_date)
    total_results = data['totalResults']
    
    # If there are no results, return early.
    if total_results == 0:
        print("No new articles available.")
        return pd.DataFrame()  # Return an empty DataFrame
 # Load URLs from both downloaded files.
    downloaded_urls = load_downloaded_articles([downloaded_file_1, downloaded_file_2])
    # Filter for new articles (by URL)
    new_articles = [article for article in data['articles'] if article['url'] not in downloaded_urls]
    
    # If there are no new articles, return early.
    if not new_articles:
        print("No new articles to download.")
        return pd.DataFrame()
    
    # Convert the new articles to a DataFrame.
    new_df = news_to_dataframe({'articles': new_articles})
    
    # Save the new article URLs.
    new_urls = [article['url'] for article in new_articles]
    # Here we choose to update one of the files (e.g., downloaded_file_2). Adjust as needed.
    store_downloaded_articles(new_urls, downloaded_file_2)
    
    return new_df

query = 'deepseek'
# Get articles from yesterday
from_date = (dt.datetime.now() - dt.timedelta(days=1)).strftime('%Y-%m-%d')
new_data = fetch_news(query, from_date)

# Show and save the new data if available
if not new_data.empty:
    print(new_data)
    new_data.to_csv('Deepseek_Day_Six.csv', index=False)


No new articles to download.


In [16]:
newsapi = NewsApiClient(api_key="####")

# Define the two downloaded files
downloaded_file_1 = 'Deepseek_Day_One.csv'
downloaded_file_2 = 'Deepseek_Day_Two.csv'
downloaded_file_3 = 'Deepseek_Day_Three.csv'
downloaded_file_4 = 'Deepseek_Day_Four.csv'
downloaded_file_5 = 'Deepseek_Day_Five.csv'

# Function to load previously downloaded article URLs from multiple files
def load_downloaded_articles(file_names: List[str]) -> set:
    urls = set()
    for file_name in file_names:
        if os.path.exists(file_name):
            df = pd.read_csv(file_name)
            urls.update(set(df['url']))
    return urls

# Function to store new article URLs in a CSV file
def store_downloaded_articles(new_urls: List[str], file_name: str):
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        df = pd.DataFrame(columns=['url'])
    
    new_df = pd.DataFrame({'url': new_urls})
    df = pd.concat([df, new_df]).drop_duplicates(subset=['url'], keep='last')
    df.to_csv(file_name, index=False)

# Function to convert NewsAPI data to a DataFrame
def news_to_dataframe(data: Dict[str, Any]) -> pd.DataFrame:
    articles = data.get('articles', [])
    processed_articles = []
    for article in articles:
        processed_article = {
            'source_id': article['source'].get('id'),
            'source_name': article['source'].get('name'),
            'author': article.get('author'),
            'title': article.get('title'),
            'description': article.get('description'),
            'url': article.get('url'),
            'urlToImage': article.get('urlToImage'),
            'publishedAt': pd.to_datetime(article.get('publishedAt')),
            'content': article.get('content')
        }
        processed_articles.append(processed_article)
    
    df = pd.DataFrame(processed_articles)
    return df

# Function to fetch new articles based on a query and date,
# while filtering out articles that have already been downloaded.
def fetch_news(query: str, from_date: str) -> pd.DataFrame:
    # Fetch data from NewsAPI
    data = newsapi.get_everything(q=query, from_param=from_date)
    total_results = data['totalResults']
    
    # If there are no results, return early.
    if total_results == 0:
        print("No new articles available.")
        return pd.DataFrame()  # Return an empty DataFrame

 # Load URLs from both downloaded files.
    downloaded_urls = load_downloaded_articles([downloaded_file_1, downloaded_file_2])
    # Filter for new articles (by URL)
    new_articles = [article for article in data['articles'] if article['url'] not in downloaded_urls]
    
    # If there are no new articles, return early.
    if not new_articles:
        print("No new articles to download.")
        return pd.DataFrame()
    
    # Convert the new articles to a DataFrame.
    new_df = news_to_dataframe({'articles': new_articles})
    
    # Save the new article URLs.
    new_urls = [article['url'] for article in new_articles]
    # Here we choose to update one of the files (e.g., downloaded_file_2). Adjust as needed.
    store_downloaded_articles(new_urls, downloaded_file_2)
    
    return new_df

query = 'deepseek'
# Get articles from yesterday
from_date = (dt.datetime.now() - dt.timedelta(days=1)).strftime('%Y-%m-%d')
new_data = fetch_news(query, from_date)

# Show and save the new data if available
if not new_data.empty:
    print(new_data)
    new_data.to_csv('Deepseek_Day_Six.csv', index=False)


No new articles to download.
